In [67]:
import tushare as ts

df = ts.pro_bar(ts_code='000063.SZ', start_date='19910101', end_date='20220923',adj='qfq',adjfactor=True)
stock_hist = df[['trade_date', 'open','close','high','low']]
stock_hist.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close'},inplace=True)
stock_hist = stock_hist[:5000]
stock_hist = stock_hist.iloc[::-1]
stock_hist.set_index('trade_date', inplace=True)
# stock_hist = stock_hist.reset_index()
# stock_hist.drop(['index'],axis=1, inplace=True)
# PINGAN.dropna()
# PINGAN.interpolate(method='linear', limit_direction='forward', axis=0)
stock_hist.head()

/var/folders/wf/l689qmr574b9nlzwszjgltk00000gn/T/ipykernel_3388/2555597064.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_hist.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close'},inplace=True)


,Open,Close,High,Low
trade_date,,,,
20010712,3.3026,3.3257,3.3809,3.3016
20010713,3.3478,3.3438,3.4331,3.3327
20010716,3.3628,3.3930,3.4281,3.3418
20010717,3.3930,3.4361,3.4482,3.3789
20010718,3.4381,3.4341,3.4632,3.4010


In [53]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(stock_hist, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

/var/folders/wf/l689qmr574b9nlzwszjgltk00000gn/T/ipykernel_3388/4084925734.py:23: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(PINGAN, SmaCross,


Row(id='6833', ...)

In [24]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/hanjie/Applications/venv2/lib/python3.8/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


Start                                     0.0
End                                    5869.0
Duration                               5869.0
Exposure Time [%]                   99.335605
Equity Final [$]                   3677.29904
Equity Peak [$]                   39489.68752
Return [%]                          -63.22701
Buy & Hold Return [%]               -6.851769
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -92.298329
Avg. Drawdown [%]                  -14.527878
Max. Drawdown Duration                 2711.0
Avg. Drawdown Duration             170.588235
# Trades                                201.0
Win Rate [%]                        44.776119
Best Trade [%]                     144.241777
Worst Trade [%]                     -54.10568
Avg. Trade [%]                    

In [54]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [55]:
# from backtesting import Backtest
# from backtesting.test import GOOG

bt = Backtest(stock_hist, SmaCross, commission=.002)

bt.run()
bt.plot()

/var/folders/wf/l689qmr574b9nlzwszjgltk00000gn/T/ipykernel_3388/1916823689.py:4: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(PINGAN, SmaCross, commission=.002)


Row(id='7780', ...)

In [56]:
def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [57]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [68]:
backtest = Backtest(stock_hist, System, commission=.002)
backtest.run()
backtest.plot()

/var/folders/wf/l689qmr574b9nlzwszjgltk00000gn/T/ipykernel_3388/2092871892.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  backtest = Backtest(stock_hist, System, commission=.002)


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [61]:
from backtesting.test import GOOG

GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800
